In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2014613,2022-01-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2014614,2022-01-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2014615,2022-01-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2014616,2022-01-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
52095,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
52097,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
52099,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
52101,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
52103,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1223118,2022-01-08,Arkansas,Arkansas,5001,4030,65.00,5000,Arkansas,AR,Arkansas,State,3017804
1223120,2022-01-09,Arkansas,Arkansas,5001,4030,65.00,5000,Arkansas,AR,Arkansas,State,3017804
1223122,2022-01-10,Arkansas,Arkansas,5001,4074,65.00,5000,Arkansas,AR,Arkansas,State,3017804
1223124,2022-01-11,Arkansas,Arkansas,5001,4166,67.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2014613,2022-01-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2014614,2022-01-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2014615,2022-01-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2014616,2022-01-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-01-12') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
722,2022-01-12,Snohomish,Washington,99803,866.00,53061,WA,County,822083,12140.26,105.34
1442,2022-01-12,Cook,Illinois,966133,12961.00,17031,IL,County,5150233,18759.02,251.66
2161,2022-01-12,Orange,California,439765,5914.00,6059,CA,County,3175692,13847.85,186.23
2879,2022-01-12,Maricopa,Arizona,969741,14064.00,4013,AZ,County,4485414,21619.88,313.55
3597,2022-01-12,Los Angeles,California,2087285,27850.00,6037,CA,County,10039107,20791.54,277.42
...,...,...,...,...,...,...,...,...,...,...,...
2012913,2022-01-12,Wheeler,Oregon,165,2.00,41069,OR,County,1332,12387.39,150.15
2013370,2022-01-12,King,Texas,30,0.00,48269,TX,County,272,11029.41,0.00
2013796,2022-01-12,Esmeralda,Nevada,74,2.00,32009,NV,County,873,8476.52,229.10
2014218,2022-01-12,Loving,Texas,54,0.00,48301,TX,County,169,31952.66,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
722,2022-01-12,Snohomish,Washington,99803,866.00,53061,WA,County,822083,12140.26,105.34,105.34,12140.26
1442,2022-01-12,Cook,Illinois,966133,12961.00,17031,IL,County,5150233,18759.02,251.66,251.66,18759.02
2161,2022-01-12,Orange,California,439765,5914.00,6059,CA,County,3175692,13847.85,186.23,186.23,13847.85
2879,2022-01-12,Maricopa,Arizona,969741,14064.00,4013,AZ,County,4485414,21619.88,313.55,313.55,21619.88
3597,2022-01-12,Los Angeles,California,2087285,27850.00,6037,CA,County,10039107,20791.54,277.42,277.42,20791.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012913,2022-01-12,Wheeler,Oregon,165,2.00,41069,OR,County,1332,12387.39,150.15,150.15,12387.39
2013370,2022-01-12,King,Texas,30,0.00,48269,TX,County,272,11029.41,0.00,0.00,11029.41
2013796,2022-01-12,Esmeralda,Nevada,74,2.00,32009,NV,County,873,8476.52,229.10,229.10,8476.52
2014218,2022-01-12,Loving,Texas,54,0.00,48301,TX,County,169,31952.66,0.00,0.00,31952.66


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1279453,2022-01-12,Hale,Alabama,3705,91.00,1065,AL,County,14651,25288.38,621.12,621.12,25288.38,1
1393823,2022-01-12,Clarke,Alabama,5479,89.00,1025,AL,County,23622,23194.48,376.77,376.77,23194.48,2
882444,2022-01-12,Clay,Alabama,3002,70.00,1027,AL,County,13235,22682.28,528.90,528.90,22682.28,3
170014,2022-01-12,Elmore,Alabama,18318,301.00,1051,AL,County,81209,22556.61,370.65,370.65,22556.61,4
1278145,2022-01-12,Bibb,Alabama,5046,95.00,1007,AL,County,22394,22532.82,424.22,424.22,22532.82,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220505,2022-01-12,Converse,Wyoming,2490,49.00,56009,WY,County,13822,18014.76,354.51,354.51,18014.76,19
1076067,2022-01-12,Johnson,Wyoming,1492,18.00,56019,WY,County,8445,17667.26,213.14,213.14,17667.26,20
1520770,2022-01-12,Lincoln,Wyoming,3274,27.00,56023,WY,County,19830,16510.34,136.16,136.16,16510.34,21
1221161,2022-01-12,Sublette,Wyoming,1580,24.00,56035,WY,County,9831,16071.61,244.13,244.13,16071.61,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
999081,2022-01-12,Lowndes,Alabama,2166,68.00,1085,AL,County,9726,22270.20,699.16,699.16,22270.20,10,1
1279453,2022-01-12,Hale,Alabama,3705,91.00,1065,AL,County,14651,25288.38,621.12,621.12,25288.38,1,2
422603,2022-01-12,Walker,Alabama,13579,392.00,1127,AL,County,63521,21377.18,617.12,617.12,21377.18,23,3
997765,2022-01-12,Crenshaw,Alabama,3065,80.00,1041,AL,County,13772,22255.30,580.89,580.89,22255.30,13,4
1078038,2022-01-12,Greene,Alabama,1493,47.00,1063,AL,County,8111,18407.10,579.46,579.46,18407.10,54,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137087,2022-01-12,Sheridan,Wyoming,6362,63.00,56033,WY,County,30485,20869.28,206.66,206.66,20869.28,11,19
1482420,2022-01-12,Uinta,Wyoming,4422,34.00,56041,WY,County,20226,21862.95,168.10,168.10,21862.95,6,20
1520770,2022-01-12,Lincoln,Wyoming,3274,27.00,56023,WY,County,19830,16510.34,136.16,136.16,16510.34,21,21
995133,2022-01-12,Albany,Wyoming,7940,44.00,56001,WY,County,38880,20421.81,113.17,113.17,20421.81,13,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,105.34,12140.26,31,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,105.34,12140.26,31,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,105.34,12140.26,31,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,105.34,12140.26,31,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,105.34,12140.26,31,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009958,2022-01-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2009959,2022-01-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2009960,2022-01-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2009961,2022-01-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1274145,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,25288.38,2,1,1.00,nan
1274146,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,25288.38,2,1,0.00,nan
1274147,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,25288.38,2,1,0.00,nan
1274148,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,621.12,25288.38,2,1,0.00,nan
1274149,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,621.12,25288.38,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546614,2022-01-08,Crook,Wyoming,1046,22.00,56011,WY,County,7584,13792.19,290.08,290.08,14095.46,11,23,0.00,1.86
1546615,2022-01-09,Crook,Wyoming,1046,22.00,56011,WY,County,7584,13792.19,290.08,290.08,14095.46,11,23,0.00,1.86
1546616,2022-01-10,Crook,Wyoming,1059,22.00,56011,WY,County,7584,13963.61,290.08,290.08,14095.46,11,23,13.00,2.79
1546617,2022-01-11,Crook,Wyoming,1064,22.00,56011,WY,County,7584,14029.54,290.08,290.08,14095.46,11,23,5.00,3.07


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
995081,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,22270.20,1,10,1.00,148.86,0.00,nan
995082,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,22270.20,1,10,0.00,136.79,0.00,nan
995083,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,22270.20,1,10,0.00,136.79,0.00,nan
995084,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,22270.20,1,10,0.00,121.21,0.00,nan
995085,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,22270.20,1,10,0.00,116.57,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417920,2022-01-08,Teton,Wyoming,7025,14.00,56039,WY,County,23464,29939.48,59.67,59.67,32637.23,23,1,0.00,101.93,0.00,0.00
417921,2022-01-09,Teton,Wyoming,7025,14.00,56039,WY,County,23464,29939.48,59.67,59.67,32637.23,23,1,0.00,101.93,0.00,0.00
417922,2022-01-10,Teton,Wyoming,7312,14.00,56039,WY,County,23464,31162.63,59.67,59.67,32637.23,23,1,287.00,114.79,0.00,0.00
417923,2022-01-11,Teton,Wyoming,7533,14.00,56039,WY,County,23464,32104.50,59.67,59.67,32637.23,23,1,221.00,126.14,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-01-12') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
505031,2022-01-12,Imperial,California,46115,812.00,6025,CA,County,181215,25447.67,448.09,448.09,25447.67,1,2,364.00,333.64,2.00,0.93
250459,2022-01-12,San Bernardino,California,456355,6198.00,6071,CA,County,2180085,20932.90,284.30,284.30,20932.90,2,4,5753.00,4466.57,14.00,10.79
1221746,2022-01-12,Tuolumne,California,9266,152.00,6109,CA,County,54478,17008.70,279.01,279.01,17008.70,3,18,115.00,71.79,1.00,0.29
77121,2022-01-12,Shasta,California,28082,500.00,6089,CA,County,180080,15594.18,277.65,277.65,15594.18,4,26,149.00,85.79,0.00,0.64
3597,2022-01-12,Los Angeles,California,2087285,27850.00,6037,CA,County,10039107,20791.54,277.42,277.42,20791.54,5,5,40373.00,31228.93,38.00,17.79
111509,2022-01-12,Stanislaus,California,103712,1501.00,6099,CA,County,550660,18834.13,272.58,272.58,18834.13,6,9,869.00,634.50,2.00,2.36
887668,2022-01-12,Inyo,California,3332,49.00,6027,CA,County,18039,18471.09,271.63,271.63,18471.09,7,12,51.00,44.14,0.00,0.07
803865,2022-01-12,Merced,California,51637,715.00,6047,CA,County,277680,18595.87,257.49,257.49,18595.87,8,10,638.00,333.36,2.00,1.21
1077980,2022-01-12,Kings,California,38807,391.00,6031,CA,County,152940,25374.00,255.66,255.66,25374.00,9,3,238.00,171.93,2.00,0.29
88595,2022-01-12,San Joaquin,California,127599,1944.00,6077,CA,County,762148,16742.02,255.07,255.07,16742.02,10,20,1552.00,1143.36,17.00,4.29


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1897108,2022-01-12,Lassen,California,8281,61.00,6035,CA,County,30573,27085.99,199.52,199.52,27085.99,19,1,37.00,15.71,0.00,0.00
505031,2022-01-12,Imperial,California,46115,812.00,6025,CA,County,181215,25447.67,448.09,448.09,25447.67,1,2,364.00,333.64,2.00,0.93
1077980,2022-01-12,Kings,California,38807,391.00,6031,CA,County,152940,25374.00,255.66,255.66,25374.00,9,3,238.00,171.93,2.00,0.29
250459,2022-01-12,San Bernardino,California,456355,6198.00,6071,CA,County,2180085,20932.90,284.30,284.30,20932.90,2,4,5753.00,4466.57,14.00,10.79
3597,2022-01-12,Los Angeles,California,2087285,27850.00,6037,CA,County,10039107,20791.54,277.42,277.42,20791.54,5,5,40373.00,31228.93,38.00,17.79
138426,2022-01-12,Tulare,California,94642,1188.00,6107,CA,County,466195,20300.95,254.83,254.83,20300.95,11,6,769.00,397.71,2.00,2.29
315297,2022-01-12,Kern,California,177220,1944.00,6029,CA,County,900202,19686.69,215.95,215.95,19686.69,14,7,1635.00,1032.36,8.00,1.86
51417,2022-01-12,Riverside,California,472262,5648.00,6065,CA,County,2470546,19115.69,228.61,228.61,19115.69,13,8,8230.00,4585.71,13.00,5.43
111509,2022-01-12,Stanislaus,California,103712,1501.00,6099,CA,County,550660,18834.13,272.58,272.58,18834.13,6,9,869.00,634.50,2.00,2.36
803865,2022-01-12,Merced,California,51637,715.00,6047,CA,County,277680,18595.87,257.49,257.49,18595.87,8,10,638.00,333.36,2.00,1.21


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
505031,2022-01-12,Imperial,California,46115,812.00,6025,CA,County,181215,25447.67,448.09,448.09,25447.67,1,2,364.00,333.64,2.00,0.93
250459,2022-01-12,San Bernardino,California,456355,6198.00,6071,CA,County,2180085,20932.90,284.30,284.30,20932.90,2,4,5753.00,4466.57,14.00,10.79
1221746,2022-01-12,Tuolumne,California,9266,152.00,6109,CA,County,54478,17008.70,279.01,279.01,17008.70,3,18,115.00,71.79,1.00,0.29
77121,2022-01-12,Shasta,California,28082,500.00,6089,CA,County,180080,15594.18,277.65,277.65,15594.18,4,26,149.00,85.79,0.00,0.64
3597,2022-01-12,Los Angeles,California,2087285,27850.00,6037,CA,County,10039107,20791.54,277.42,277.42,20791.54,5,5,40373.00,31228.93,38.00,17.79
111509,2022-01-12,Stanislaus,California,103712,1501.00,6099,CA,County,550660,18834.13,272.58,272.58,18834.13,6,9,869.00,634.50,2.00,2.36
887668,2022-01-12,Inyo,California,3332,49.00,6027,CA,County,18039,18471.09,271.63,271.63,18471.09,7,12,51.00,44.14,0.00,0.07
803865,2022-01-12,Merced,California,51637,715.00,6047,CA,County,277680,18595.87,257.49,257.49,18595.87,8,10,638.00,333.36,2.00,1.21
1077980,2022-01-12,Kings,California,38807,391.00,6031,CA,County,152940,25374.00,255.66,255.66,25374.00,9,3,238.00,171.93,2.00,0.29
88595,2022-01-12,San Joaquin,California,127599,1944.00,6077,CA,County,762148,16742.02,255.07,255.07,16742.02,10,20,1552.00,1143.36,17.00,4.29


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,2,4.00,14.43
1,03/21/20,Imperial,4,2.21,2,0.00,13.93
2,03/22/20,Imperial,4,2.21,2,0.00,13.93
3,03/23/20,Imperial,4,2.21,2,0.00,13.93
4,03/24/20,Imperial,9,4.97,2,5.00,14.29
...,...,...,...,...,...,...,...
6652,01/08/22,Lassen,8191,26791.61,1,32.00,9.29
6653,01/09/22,Lassen,8191,26791.61,1,0.00,9.29
6654,01/10/22,Lassen,8191,26791.61,1,0.00,9.29
6655,01/11/22,Lassen,8244,26964.97,1,53.00,13.07


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6699,01/08/22,San Joaquin,1916.00,251.39,10,0.00,3.14
6700,01/09/22,San Joaquin,1916.00,251.39,10,0.00,3.14
6701,01/10/22,San Joaquin,1927.00,252.84,10,11.00,3.93
6702,01/11/22,San Joaquin,1927.00,252.84,10,0.00,3.64


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)